In [1]:
from astropy.io import fits
import clevar
from clevar.match_metrics.recovery import ClCatalogFuncs as r_cf
from clevar.match_metrics.distances import ClCatalogFuncs as d_cf
from clevar.match_metrics.scaling import ClCatalogFuncs as s_cf
import clevar.match_metrics.scaling as scaling
import pyccl as ccl
import pandas as pd
from astropy.table import Table
import astropy
from matplotlib import pyplot as plt
import numpy as np
from numcosmo_py import Ncm, Nc
from numcosmo_py.external.pyssc import pyssc as PySSC
Ncm.cfg_init()

In [3]:
hdul_halos = fits.open("../halos/halos.fits")
hdul_ame = fits.open("richness_mcut_SNR 12_ame.fits")

logMmin = 14.0
logMmax = 15

table_halos = hdul_halos[1].data
table_ame = hdul_ame[1].data
table_halos = table_halos[(table_halos['upid'] == -1)  &  (table_halos['Mvir'] >= 10**logMmin) & (table_halos['Mvir'] <= 10**logMmax)]

hdu = fits.BinTableHDU(data=table_halos)
hdu.writeto('ame_halos.fits' , overwrite=True)

table_halos['Mvir'] = table_halos['Mvir']/0.6774

print(table_halos.size)
print(table_ame.size)

#print(table_ame.columns)

print("DEC max")
print(table_ame['DEC'].max())
print(table_halos['DEC'].max())
print('\n')
print("DEC min")
print(table_ame['DEC'].min())
print(table_halos['DEC'].min())
print('\n')
print("RA max")
print(table_ame['RA'].max())
print(table_halos['RA'].max())
print('\n')
print("RA min")
print(table_ame['RA'].min())
print(table_halos['RA'].min())
print('\n')
print("z max")
print(table_ame['z'].max())
print(table_halos['redshift_R'].max())
print('\n')
print("z min")
print(table_ame['z'].min())
print(table_halos['redshift_R'].min())

#display(table_halos['RA'].duplicates())
#display(table_halos[14600])

23066
23751
DEC max
69.987462659708
69.99612426757812


DEC min
20.00571528380776
20.00299072265625


RA max
229.99483480331588
229.99781799316406


RA min
110.00560235552096
110.01350402832031


z max
0.6587940911577925
0.6646252277727887


z min
0.0462011092646255
0.016908477103261287


In [5]:
catalog_halos = clevar.ClCatalog('Halo catalog',ra=table_halos['RA'] ,dec=table_halos['DEC'] 
                                ,z=table_halos['redshift_R']) #, mass=table_halos['Mvir']

catalog_ame = clevar.ClCatalog('Detection catalog', ra=table_ame['RA'], dec=table_ame['DEC']
                             , z=table_ame['z'] ,z_err=table_ame['z_err'] ) #mass=table_ame['R_ame']  , mass_err=table_ame['R_ame_err']

/global/u2/l/lettieri/miniforge3/envs/numcosmo_developer/lib/python3.12/site-packages/clevar-0.15.3-py3.12.egg/clevar/catalog/tagdata.py:278: UserWarning: id column missing, additional one is being created.
  warnings.warn(f"{id_name} column missing, additional one is being created.")


In [4]:
cosmo_astropy = clevar.cosmology.CCLCosmology(H0=67.74, Omega_b0=0.0486, Omega_dm0=0.3089 - 0.0486, Omega_k0=0)
mt = clevar.match.ProximityMatch()
mt.prep_cat_for_match(catalog_ame ,  delta_z=0.01, match_radius='1 Mpc', cosmo=cosmo_astropy)
mt.prep_cat_for_match(catalog_halos ,  delta_z=0.01, match_radius='1 Mpc', cosmo=cosmo_astropy)
match_config = {
    'type': 'cross', # options are cross, cat1, cat2
    'which_radius': 'self', # Case of radius to be used, can be: cat1, cat2, min, max
    'preference': 'redshift_proximity', # options are more_massive, angular_proximity or redshift_proximity
    'catalog1': {'delta_z':0.01,
                'match_radius': '1 Mpc',
                 'n_delta_z':1.0
                },
    'catalog2': {'delta_z':0.01,
                'match_radius': '1 Mpc',
                 'n_delta_z':1.0
                }
}

mt.match_from_config(catalog_halos, catalog_ame, match_config, cosmo=cosmo_astropy)
mt.save_matches(catalog_halos, catalog_ame, out_dir='temp', overwrite=True)

## Prep mt_cols
### Prep z_cols
* zmin|zmax from config value
### Prep ang_cols
* ang radius from set scale
## Prep mt_cols
### Prep z_cols
* zmin|zmax from config value
### Prep ang_cols
* ang radius from set scale

## ClCatalog 1
## Prep mt_cols
### Prep z_cols
* zmin|zmax from config value
### Prep ang_cols
* ang radius from set scale

## ClCatalog 2
## Prep mt_cols
### Prep z_cols
* zmin|zmax from config value
### Prep ang_cols
* ang radius from set scale

## Multiple match (catalog 1)
Finding candidates (Halo catalog)
* 11,251/23,066 objects matched.

## Multiple match (catalog 2)
Finding candidates (Detection catalog)
* 11,081/23,751 objects matched.

## Finding unique matches of catalog 1
Unique Matches (Halo catalog)
* 11,074/23,066 objects matched.

## Finding unique matches of catalog 2
Unique Matches (Detection catalog)
* 11,076/23,751 objects matched.
Cross Matches (Halo catalog)
* 11,068/23,066 objects matched.
Cross Matches (Detection catalog)
* 11,068/23,751 objects matc

In [4]:
catalog_halos = clevar.ClCatalog('Halo catalog',ra=table_halos['RA'] ,dec=table_halos['DEC'] 
                                ,z=table_halos['redshift_R']) #, mass=table_halos['Mvir']

catalog_ame = clevar.ClCatalog('Detection catalog', ra=table_ame['RA'], dec=table_ame['DEC']
                             , z=table_ame['z'] , ) #z_err=table_ame['z_err'] , , mass=table_ame['R_ame']  , mass_err=table_ame['R_ame_err']
mt = clevar.match.ProximityMatch()
mt.load_matches(catalog_halos, catalog_ame, out_dir='temp')

halos_match, detections_match = clevar.match.get_matched_pairs(catalog_halos, catalog_ame, 'cross')

/global/u2/l/lettieri/miniforge3/envs/numcosmo_developer/lib/python3.12/site-packages/clevar-0.15.3-py3.12.egg/clevar/catalog/tagdata.py:278: UserWarning: id column missing, additional one is being created.
  warnings.warn(f"{id_name} column missing, additional one is being created.")


Halo catalog
    * ClEvar used in matching: 0.15.3
 * Total objects:    23,066
 * multiple (self):  11,251
 * multiple (other): 11,254
 * unique (self):    11,074
 * unique (other):   11,076
 * cross:            11,068

Detection catalog
    * ClEvar used in matching: 0.15.3
 * Total objects:    23,751
 * multiple (self):  11,081
 * multiple (other): 11,079
 * unique (self):    11,076
 * unique (other):   11,074
 * cross:            11,068


In [1]:
from numcosmo_py import Ncm, Nc
import numpy as np
from numcosmo_py import sky_match 
from numcosmo_py.external.pyssc import pyssc as PySSC
from pathlib import Path
Ncm.cfg_init()

area = 4109.3
cosmo = Nc.HICosmoDEXcdm()
cosmo.omega_x2omega_k()
cosmo.param_set_by_name("H0", 67.74)
cosmo.param_set_by_name("Omegab", 0.0486)
cosmo.param_set_by_name("Omegac",0.3089 - 0.0486) #0.2603
cosmo.param_set_by_name("Omegak", 0.00)
cosmo.param_set_by_name("w", -1.0) #-1.0


prim = Nc.HIPrimPowerLaw.new()
prim.param_set_by_name("ln10e10ASA",3.0116707481421923)
prim.param_set_by_name("n_SA", 0.9667)


reion = Nc.HIReionCamb.new()

cosmo.add_submodel(prim)
cosmo.add_submodel(reion)


path_halo = Path("ame_halos.fits")
halo_coordinates = {"RA":"RA" , "DEC":"DEC" , "z":"redshift_R"}
halo_properties  = {"Mvir":"mass1" ,"RA":"RA1"}
path_detections  = Path("richness_mcut_SNR 12_ame.fits")
detections_coordinates =  {"RA":"RA" , "DEC":"DEC" , "z":"z"}
detections_properties  = {"R_ame":"mass2" , "z_err":"z_err2"}


#detections = sky_match.SkyMatch(query_catalog_path=path_detections, query_coordinates=detections_coordinates, query_properties=detections_properties
#                                , match_catalog_path=path_halo, match_coordinates=halo_coordinates , match_properties=halo_properties)


halos = sky_match.SkyMatch(query_catalog_path=path_halo, query_coordinates=halo_coordinates , query_properties=halo_properties ,match_catalog_path=path_detections,
                   match_coordinates=detections_coordinates,match_properties=detections_properties)




#halos = sky_match.SkyMatch(query_catalog_path=path_halo, query_coordinates=halo_coordinates ,match_catalog_path=path_detections,
#                   match_coordinates=detections_coordinates)

halos_matched, detections_matched, best_halos, best_detections , cross = halos.cross_match_2d(cosmo=cosmo  ,matching_distance1=1 ,matching_distance2=1,n_nearest_neighbours1=100,n_nearest_neighbours2=100 
                                                                                   ,match_dist_3d=True , delta_z1=0.01,delta_z2=0.01 , n_delta_z1=1,n_delta_z2=1 ,
                                                                                 selection_criteria1='redshift_proximity' , selection_criteria2='redshift_proximity',
                                                                                  use_zerr1 =False      ,use_zerr2 = False ,which_radius1="query_radius" , which_radius2="query_radius")

100%|██████████| 23066/23066 [00:08<00:00, 2719.47it/s]



        Number of objects matching: 23066
        Number of objects in the target catalog: 23751
        Number of unmatched objects: 11816
        Number of matched objects: 11250
        


100%|██████████| 23751/23751 [00:08<00:00, 2886.08it/s]



        Number of objects matching: 23751
        Number of objects in the target catalog: 23066
        Number of unmatched objects: 12670
        Number of matched objects: 11081
        


100%|██████████| 23066/23066 [00:10<00:00, 2100.71it/s]


        Number of cross matched objects: 11070
        


In [8]:
display(len(halos_matched))
display(len(detections_matched))
display(len(cross))
halos_number = 0
#display(cross)
detections_number = 0
for i in range(len(halos_matched['ID'])):
    if len(halos_matched['ID_matched'][i]) > 0:
        halos_number +=1

for i in range(len(detections_matched['ID'])):
    if len(detections_matched['ID_matched'][i]) > 0:
        detections_number +=1
print(halos_number)
print(detections_number)

catalog_halos = clevar.ClCatalog('Halo catalog',ra=table_halos['RA'] ,dec=table_halos['DEC'] 
                                ,z=table_halos['redshift_R']) #, mass=table_halos['Mvir']

catalog_ame = clevar.ClCatalog('Detection catalog', ra=table_ame['RA'], dec=table_ame['DEC']
                             , z=table_ame['z'] , ) #z_err=table_ame['z_err'] , , mass=table_ame['R_ame']  , mass_err=table_ame['R_ame_err']
mt = clevar.match.ProximityMatch()
mt.load_matches(catalog_halos, catalog_ame, out_dir='temp')

halos_match, detections_match = clevar.match.get_matched_pairs(catalog_halos, catalog_ame, 'self')
#0.0003657

23066

23751

11070

/global/u2/l/lettieri/miniforge3/envs/numcosmo_developer/lib/python3.12/site-packages/clevar-0.15.3-py3.12.egg/clevar/catalog/tagdata.py:278: UserWarning: id column missing, additional one is being created.
  warnings.warn(f"{id_name} column missing, additional one is being created.")


11250
11081
Halo catalog
    * ClEvar used in matching: 0.15.3
 * Total objects:    23,066
 * multiple (self):  11,251
 * multiple (other): 11,254
 * unique (self):    11,074
 * unique (other):   11,076
 * cross:            11,068

Detection catalog
    * ClEvar used in matching: 0.15.3
 * Total objects:    23,751
 * multiple (self):  11,081
 * multiple (other): 11,079
 * unique (self):    11,076
 * unique (other):   11,074
 * cross:            11,068


In [3]:
#display(halos_matched[ids])
#display(catalog_halos[ids])
display(detections_matched[372])
display(halos_matched[1111])

ID,RA,DEC,z,ID_matched,RA_matched,DEC_matched,z_matched,distances,mass2,z_err2,mass1,RA1
int64,float64,float64,float64,int64[100],float64[100],float64[100],float64[100],float64[100],float64,float64,float64[100],float64[100]
372,198.159633637546,55.81167992400343,0.1931849769991934,1111 .. 8784,198.1243896484375 .. 195.29347229003906,55.81852722167969 .. 57.56620788574219,0.194790192623097 .. 0.426504179211944,0.0003656952328338415 .. 0.04112952717814077,84.0,0.0031849769991934,141899998000000.0 .. 306299988000000.0,198.1243896484375 .. 195.29347229003906


ID,RA,DEC,z,ID_matched,RA_matched,DEC_matched,z_matched,distances,mass1,RA1,mass2,z_err2
int64,float64,float64,float64,int64[100],float64[100],float64[100],float64[100],float64[100],float64,float64,float64[100],float64[100]
1111,198.1243896484375,55.81852722167969,0.194790192623097,13770 .. 13095,198.159633637546 .. 196.1824424589641,55.81167992400343 .. 57.73292192526432,0.1545603205774501 .. 0.3612993701840821,0.0003656952328338415 .. 0.0382205563698647,141899998000000.0,198.1243896484375,94.52 .. 40.2,0.0045603205774501 .. 0.001299370184082


In [10]:
dist = Nc.Distance.new(3.0)
dist.prepare(cosmo)
RH_Mpc = cosmo.RH_Mpc()
print(dist.angular_diameter(cosmo, 0.1545603205774501) * RH_Mpc * 0.0003656952328338415)
print(dist.angular_diameter(cosmo, 0.194790192623097) * RH_Mpc * 0.0003656952328338415)
print(dist.angular_diameter(cosmo, 0.1931849769991934) * RH_Mpc * 0.0003656952328338415)

0.20870139288905715
0.2515896399689881
0.24995442784729804


In [2]:
print(halos_matched['ID_matched'][1111])
detections_matched['ID_matched'][372]

[]


array([1111])